# Классификаторы голосования 

Очень простой способ создать еще лучший классификатор — агрегировать прогнозы каждого классификатора: класс, набравший наибольшее количество голосов, является прогнозом ансамбля. Этот классификатор большинства голосов называется классификатором жесткого голосования

In [16]:
from sklearn.datasets import make_moons 
from sklearn.ensemble import RandomForestClassifier, VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC 
 
X, y = make_moons(n_samples=500, noise=0.30, random_state=42) # взяли набор данных moons 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 
 
voting_clf = VotingClassifier( # класс, проводящий голосование среди трех моделей
    estimators=[ 
        ('lr', LogisticRegression(random_state=42)), 
        ('rf', RandomForestClassifier(random_state=42)), 
        ('svc', SVC(random_state=42)) 
    ] 
) 
voting_clf.fit(X_train, y_train) # обучаем класс

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

Когда вы подбираете VotingClassifier, он клонирует каждую оценку и подбирает клоны. Исходные оценщики доступны через estimators атрибут, а подобранные клоны доступны через estimators_ атрибут. Если вы предпочитаете словарь, а не список, вы можете использовать named_estimators или named_estimators_ вместо него. Для начала давайте посмотрим на точность каждого подобранного классификатора на тестовом наборе: 

In [17]:
for name, clf in voting_clf.named_estimators_.items(): 
    print(name, "=", clf.score(X_test, y_test))

lr = 0.864
rf = 0.896
svc = 0.896


In [18]:
voting_clf.predict(X_test[:1]) 

array([1], dtype=int64)

In [19]:
[clf.predict(X_test[:1]) for clf in voting_clf.estimators_]

[array([1], dtype=int64), array([1], dtype=int64), array([0], dtype=int64)]

In [20]:
voting_clf.score(X_test, y_test) # Классификатор голосования превосходит все индивидуальные классификаторы 

0.912

Если все классификаторы способны оценивать вероятности классов (т. е. если все они имеют метод predict_proba()), то вы можете указать Scikit-Learn предсказать класс с наибольшей вероятностью класса, усредненной по всем отдельным классификаторам. Это называется мягким голосованием . 

In [21]:
# Все, что вам нужно сделать, это установить для гиперпараметра классификатора голосования voting значение "soft" и убедиться,
# что все классификаторы могут оценивать вероятности классов
voting_clf.voting = "soft" 
voting_clf.named_estimators["svc"].probability = True #добавит метод predict_proba() для svc
voting_clf.fit(X_train, y_train) 
voting_clf.score(X_test, y_test) 


0.92

# Упаковка и склеивание 

Один из способов получить разнообразный набор классификаторов — использовать очень разные алгоритмы обучения, как только что обсуждалось. Другой подход состоит в том, чтобы использовать один и тот же алгоритм обучения для каждого предиктора, но обучать их на разных случайных подмножествах обучающей выборки. 

Как только все предикторы обучены, ансамбль может сделать прогноз для нового экземпляра, просто объединив прогнозы всех предикторов. Функция агрегирования обычно представляет собой статистический режим классификации (т. е. наиболее частое предсказание, как и в классификаторе с жестким голосованием) или среднее значение для регрессии. Каждый отдельный предиктор имеет более высокое смещение, чем если бы он был обучен на исходном обучающем наборе, но агрегирование уменьшает как смещение, так и дисперсию . на исходном тренировочном наборе. 

In [25]:
# Scikit-Learn предлагает простой API как для упаковки, так и для вставки: BaggingClassifier класс (или BaggingRegressor для регрессии).
from sklearn.ensemble import BaggingClassifier 
from sklearn.tree import DecisionTreeClassifier 

# Следующий код обучает ансамбль из 500 классификаторов дерева решений: 
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, oob_score=True, # oob_score=True - запрос на автоматическую 
                            #оценку OOB после обучения
                            max_samples=100, n_jobs=-1, random_state=42) 
# каждый обучается на 100 обучающих экземплярах, случайно выбранных из обучающего набора с заменой (это пример упаковки,
# но если вместо этого вы хотите использовать вставку, просто установите bootstrap=False).
bag_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, oob_score=True, random_state=42)

In [26]:
bag_clf.score(X_test, y_test) 

0.904

In [27]:
bag_clf.oob_score_

0.9253333333333333

Согласно этой оценке OOB, BaggingClassifier, вероятно, обеспечит точность около 92.5% на тестовом наборе. Давайте проверим это: 

In [29]:
from sklearn.metrics import accuracy_score 
y_pred = bag_clf.predict(X_test) 
accuracy_score(y_test, y_pred)
# Мы получаем 90.4% точности на тесте. Оценка OOB была слишком оптимистичной, завышенной чуть более чем на 2%. 

0.904

Функция принятия решения OOB для каждого экземпляра обучения также доступна через oob_decision_function_атрибут. Поскольку у базовой оценки есть predict_proba()метод, функция принятия решения возвращает вероятности класса для каждого экземпляра обучения. Например, оценка OOB оценивает, что первый обучающий экземпляр имеет вероятность 67,6% принадлежности к положительному классу и вероятность 32,4% принадлежности к отрицательному классу: 

In [30]:
bag_clf.oob_decision_function_[:3] # вероятность для первых трех экземпляров

array([[0.35579515, 0.64420485],
       [0.43513514, 0.56486486],
       [1.        , 0.        ]])

Класс BaggingClassifier также поддерживает выборку функций. Выборка управляется двумя гиперпараметрами: max_features и  bootstrap_features. Они работают так же, как max_samples и bootstrap, но для выборки объектов вместо выборки экземпляров. Таким образом, каждый предиктор будет обучаться на случайном подмножестве входных признаков. 

Функции выборки приводят к еще большему разнообразию предикторов, заменяя немного большее смещение на более низкую дисперсию. 

# Случайные леса

Вместо того, чтобы строить a BaggingClassifier и передавать ему a DecisionTreeClassifier, можно использовать RandomForestClassifier

In [31]:
from sklearn.ensemble import RandomForestClassifier 
 
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, # код обучает классификатор 
#случайного леса с 500 деревьями, каждое из которых ограничено 16 конечными узлами, используя все доступные ядра ЦП
                                 n_jobs=-1, random_state=42) 
rnd_clf.fit(X_train, y_train) 
 
y_pred_rf = rnd_clf.predict(X_test) 

## Дополнительные деревья

Когда вы выращиваете дерево в случайном лесу, в каждом узле для разделения рассматривается только случайное подмножество объектов (как обсуждалось ранее). Можно сделать деревья еще более случайными, если использовать случайные пороги для каждой функции, а не искать наилучшие возможные пороги (как это делают обычные деревья решений). Для этого просто установите splitter="random" при создании файла DecisionTreeClassifier. 

Лес таких чрезвычайно случайных деревьев называется ансамблем чрезвычайно случайных деревьев ⁠ 12 (или для краткости экстра-деревьев ). Еще раз, этот метод меняет большую предвзятость на меньшую дисперсию. Это также делает классификаторы дополнительных деревьев намного быстрее обучаемыми, чем обычные случайные леса, потому что поиск наилучшего возможного порога для каждой функции в каждом узле является одной из самых трудоемких задач выращивания дерева. 

Вы можете создать классификатор дополнительных деревьев, используя ExtraTreesClassifier класс Scikit-Learn.

Трудно заранее сказать, RandomForestClassifier будет ли a работать лучше или хуже, чем ExtraTreesClassifier. Как правило, единственный способ узнать это — попробовать оба варианта и сравнить их с помощью перекрестной проверки. 

## Важность функций

In [34]:
from sklearn.datasets import load_iris 
iris = load_iris(as_frame=True) # датасет ирисов
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42) 
rnd_clf.fit(iris.data, iris.target) # обучаем модель случайный лес
for score, name in zip(rnd_clf.feature_importances_, iris.data.columns): 
    print(round(score, 2), name) # получаем оценку важности признаков

0.11 sepal length (cm)
0.02 sepal width (cm)
0.44 petal length (cm)
0.42 petal width (cm)


# Адаптивное повышения (АдаБуст)

Повышение (первоначально называвшееся повышением гипотезы ) относится к любому ансамблевому методу, который может объединить несколько слабых учеников в одного сильного ученика. Общая идея большинства методов бустинга состоит в последовательном обучении предикторов, каждый из которых пытается исправить своего предшественника.

При обучении классификатора AdaBoost алгоритм сначала обучает базовый классификатор (например, дерево решений) и использует его для прогнозирования обучающего набора. Затем алгоритм увеличивает относительный вес ошибочно классифицированных обучающих примеров. Затем он обучает второй классификатор, используя обновленные веса, и снова делает прогнозы на обучающем наборе, обновляет веса экземпляров и т. д.

У этого метода последовательного обучения есть один важный недостаток: обучение нельзя распараллелить, поскольку каждый предиктор можно обучить только после того, как предыдущий предиктор был обучен и оценен. В результате он не масштабируется так же хорошо, как упаковка или вставка. 

Scikit-Learn использует мультиклассовую версию AdaBoost под названием SAMME ⁠ (что означает поэтапное аддитивное моделирование с использованием функции мультиклассовых экспоненциальных потерь ).Когда есть только два класса, SAMME эквивалентен AdaBoost. Если предикторы могут оценивать вероятности класса (т. е. если у них есть predict_proba()метод), Scikit-Learn может использовать вариант SAMME, называемый SAMME.R ( R означает «Реальный»), который опирается на вероятности класса, а не на предсказания, и обычно работает лучше. 

In [35]:
from sklearn.ensemble import AdaBoostClassifier 
# Следующий код обучает классификатор AdaBoost на основе 30 пней решений:
ada_clf = AdaBoostClassifier( 
    DecisionTreeClassifier(max_depth=1), n_estimators=30, 
    learning_rate=0.5, random_state=42) 
ada_clf.fit(X_train, y_train) 

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=30, random_state=42)

# Повышение градиента

Еще одним очень популярным алгоритмом повышения является повышение градиента.⁠ Как и в случае с AdaBoost, повышение градиента работает путем последовательного добавления предикторов в ансамбль, каждый из которых корректирует своего предшественника. Однако вместо настройки весов экземпляров на каждой итерации, как это делает AdaBoost, этот метод пытается подогнать новый предиктор к остаточным ошибкам, сделанным предыдущим предиктором. 

Пройдемся по простому пример регрессии с использованием деревьев решений в качестве базовых предикторов; это называется повышением градиентного дерева или деревьями регрессии с градиентным усилением (GBRT).Во-первых, давайте сгенерируем зашумленный квадратичный набор данных и подгоним DecisionTreeRegressor к нему: 

In [36]:
import numpy as np 
from sklearn.tree import DecisionTreeRegressor 
 
np.random.seed(42) 
X = np.random.rand(100, 1) - 0.5 
y = 3 * X[:, 0] ** 2 + 0.05 * np.random.randn(100)  # y = 3x² + Gaussian noise 
 
tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42) # обучаем модель
tree_reg1.fit(X, y) 

# Затем мы обучим второго DecisionTreeRegressor на остаточных ошибках, сделанных первым предиктором: 

y2 = y - tree_reg1.predict(X) 
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=43) 
tree_reg2.fit(X, y2) 

#А затем мы обучим третий регрессор на остаточных ошибках, сделанных вторым предиктором: 
y3 = y2 - tree_reg2.predict(X) 
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=44) 
tree_reg3.fit(X, y3) 

# Теперь у нас есть ансамбль из трех деревьев. Он может делать прогнозы для нового экземпляра, просто складывая прогнозы всех деревьев: 
X_new = np.array([[-0.4], [0.], [0.5]]) 
sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

array([0.49484029, 0.04021166, 0.75026781])

Вы можете использовать класс Scikit-Learn GradientBoostingRegressor для более простого обучения ансамблей GBRT (есть также GradientBoostingClassifier класс для классификации). Как и RandomForestRegressor класс, он имеет гиперпараметры для управления ростом деревьев решений (например, max_depth, min_samples_leaf), а также гиперпараметры для управления обучением ансамбля, такие как количество деревьев ( n_estimators). Следующий код создает тот же ансамбль, что и предыдущий: 

In [37]:
from sklearn.ensemble import GradientBoostingRegressor 
 
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, 
                                 learning_rate=1.0, random_state=42) 
gbrt.fit(X, y) 

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

Гиперпараметр learning_rate масштабирует вклад каждого дерева. Если вы установите низкое значение, например 0.05, вам потребуется больше деревьев в ансамбле, чтобы соответствовать обучающему набору, но прогнозы, как правило, будут лучше обобщаться. Это метод регуляризации, называемый усадкой.

Чтобы найти оптимальное количество деревьев, вы можете выполнить перекрестную проверку с помощью GridSearchCV или RandomizedSearchCV, как обычно, но есть более простой способ: если вы установите гиперпараметр n_iter_no_change в целое значение, скажем, 10, то GradientBoostingRegressor во время обучения автоматически перестанет добавлять больше деревьев, если он видит, что последние 10 деревьев не помогли. Это просто ранняя остановка (представленная в главе 4 ), но с небольшим терпением: она допускает отсутствие прогресса в течение нескольких итераций до остановки. Давайте обучим ансамбль, используя раннюю остановку: 

In [38]:
gbrt_best = GradientBoostingRegressor( 
    max_depth=2, learning_rate=0.05, n_estimators=500, 
    n_iter_no_change=10, random_state=42) 
gbrt_best.fit(X, y) 

GradientBoostingRegressor(learning_rate=0.05, max_depth=2, n_estimators=500,
                          n_iter_no_change=10, random_state=42)

In [40]:
gbrt_best.n_estimators_ # количество обучающих оценщиков 92 вместо 500, благодаря ранней остановке n_iter_no_change


92

- Если n_iter_no_change установлено, fit()метод автоматически разбивает обучающий набор на меньший обучающий набор и проверочный набор: это позволяет оценивать производительность модели каждый раз, когда добавляется новое дерево. Размер проверочного набора управляется гиперпараметром validation_fraction, который по умолчанию равен 10%.


- Класс GradientBoostingRegressor также поддерживает subsample гиперпараметр, который указывает долю обучающих экземпляров, которые будут использоваться для обучения каждого дерева. Например, если subsample=0.25, то каждое дерево обучается на 25% обучающих экземпляров, выбранных случайным образом.

- Гиперпараметр tol определяет максимальное улучшение производительности, которое по-прежнему считается незначительным. По умолчанию он равен 0,0001.

# Повышение градиента на основе гистограммы (HGB)

Работает путем объединения входных функций, заменяя их целыми числами. Количество бинов контролируется гиперпараметром max_bins, который по умолчанию равен 255 и не может быть установлен выше этого значения. Способ построения бинов устраняет необходимость сортировки признаков при обучении каждого дерева.


Scikit-Learn предоставляет два класса для HGB: HistGradientBoostingRegressor и  HistGradientBoostingClassifier. Они похожи на GradientBoostingRegressor и GradientBoostingClassifier с несколькими заметными отличиями: 

- Ранняя остановка активируется автоматически, если количество экземпляров превышает 10 000. Вы можете всегда включать или выключать раннюю остановку, установив для early_stopping гиперпараметра значение True или False. 

- Подвыборка не поддерживается. 

- n_estimators переименовывается в max_iter. 

- Единственными гиперпараметрами дерева решений, которые можно настроить, являются max_leaf_nodes, min_samples_leaf и max_depth. 

**Классы HGB также имеют две приятные особенности: они поддерживают как категориальные признаки, так и пропущенные значения. Это немного упрощает предварительную обработку. Однако категориальные признаки должны быть представлены целыми числами в диапазоне от 0 до числа меньше max_bins. Вы можете использовать OrdinalEncoder для этого.**

In [46]:
# Загрузим данные о Калифорнии
from pathlib import Path 
import tarfile 
import urllib.request
import pandas as pd
def load_housing_data(): 
    tarball_path = Path("datasets/housing.tgz") 
    if not tarball_path.is_file(): 
        Path("datasets").mkdir(parents=True, exist_ok=True) 
        url = "https://github.com/ageron/data/raw/main/housing.tgz" 
        urllib.request.urlretrieve(url, tarball_path) 
        with tarfile.open(tarball_path) as housing_tarball: 
            housing_tarball.extractall(path="datasets") 
    return pd.read_csv(Path("datasets/housing/housing.csv")) 
 
housing = load_housing_data()

In [47]:
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing_labels = train_set["median_house_value"]
housing = train_set.drop("median_house_value", axis=1)

In [48]:
from sklearn.pipeline import make_pipeline 
from sklearn.compose import make_column_transformer 
from sklearn.ensemble import HistGradientBoostingRegressor 
from sklearn.preprocessing import OrdinalEncoder 
 
hgb_reg = make_pipeline( 
    make_column_transformer((OrdinalEncoder(), ["ocean_proximity"]), # кодируем категориальный столбец
                            remainder="passthrough"), 
    HistGradientBoostingRegressor(categorical_features=[0], random_state=42) 
    # categorical_features должны быть установлены категориальные индексы столбцов
) 
hgb_reg.fit(housing, housing_labels) # используем данные о жилье в калифорнии

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['ocean_proximity'])])),
                ('histgradientboostingregressor',
                 HistGradientBoostingRegressor(categorical_features=[0],
                                               random_state=42))])

# Укладка

Агрегирование прогнозов происходит с использованием предиктора смешивания (блендера).На самом деле можно обучить таким образом несколько различных блендеров (например, один с использованием линейной регрессии, другой с использованием регрессии случайного леса), чтобы получить целый слой блендеров, а затем добавить еще один блендер поверх него, чтобы получить окончательный прогноз.Scikit-Learn предоставляет два класса для объединения ансамблей: StackingClassifier и StackingRegressor.

In [41]:
from sklearn.ensemble import StackingClassifier 
X, y = make_moons(n_samples=500, noise=0.30, random_state=42) # взяли набор данных moons 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 
 
stacking_clf = StackingClassifier( 
    estimators=[ 
        ('lr', LogisticRegression(random_state=42)), 
        ('rf', RandomForestClassifier(random_state=42)), 
        ('svc', SVC(probability=True, random_state=42)) 
    ], 
    final_estimator=RandomForestClassifier(random_state=43), 
    cv=5  # Чтобы обучить блендер, сначала нужно создать набор для обучения смешиванию, необходимо исп-ть кросс-валидацию
) 
stacking_clf.fit(X_train, y_train) 

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=43))

In [43]:
stacking_clf.score(X_test, y_test) # классификатор стекирования превзошел классификатор голосования

0.928

Если не предоставить окончательный класс-блендер,  то StackingClassifier будет использоваться LogisticRegression по умолчанию и  StackingRegressor будет использоваться RidgeCV.

В заключение, ансамблевые методы универсальны, эффективны и довольно просты в использовании. Случайные леса, AdaBoost и GBRT являются одними из первых моделей, которые вы должны протестировать для большинства задач машинного обучения, и они особенно хороши для разнородных табличных данных. Более того, поскольку они требуют очень небольшой предварительной обработки, они отлично подходят для быстрого создания и запуска прототипа. Наконец, ансамблевые методы, такие как классификаторы голосования и классификаторы стекирования, могут помочь довести производительность вашей системы до предела. 